In [76]:
import os
import numpy as np
import cadquery as cq
from microgen import *

from jupyter_cadquery import show, set_defaults, open_viewer
from jupyter_cadquery.replay import enable_replay, disable_replay, reset_replay, get_context, replay, Replay, _CTX
from jupyter_cadquery.cad_objects import to_assembly

cv = open_viewer("Box", cad_width=640, height=480)

set_defaults(reset_camera=True, show_parent=False, axes=True, axes0=True)

# Shapes available

In [ ]:
shapes = {"box":             {"dim_x": 0.8, "dim_y": 0.8, "dim_z": 0.8},
          "sphere":          {"radius": 0.5},
          "capsule":         {"height": 0.5, "radius": 0.3},
          "cylinder":        {"height": 0.5, "radius": 0.5},
          "ellipsoid":       {"a_x": 0.5, "a_y": 0.25, "a_z": 0.3},
          "extrudedpolygon": {"listCorners": [(0.5, 0), (0.25, 0.44), (-0.25, 0.44), 
                                              (-0.5, 0), (-0.25, -0.44), (0.25, -0.44), (0.5, 0)],
                              "height": 0.5}} #, "polyhedron":      {"dic": }}


assembly = cq.Assembly()

n_col = 3
n_row = np.ceil(len(shapes)/n_col)
i = 0
for shape, param_geom in shapes.items():
    i_x = i%n_col
    i_y = i//n_col
    elem = BasicGeometry(number=i, shape=shape,
                         xc= 1.2*(i_x - 0.5*(n_col - 1)), 
                         yc=-1.2*(i_y - 0.5*(n_row - 1)), 
                         zc=0,
                         psi=90, theta=90, phi=90,
                         param_geom=param_geom)
    assembly.add(elem.generate())
    i = i + 1

compound = assembly.toCompound()
show(compound)

# Honeycomb

In [ ]:
l = 2.5 # side in mm of the hexagon
h = 2.5 # height in mm of the hexagon
theta = 30*np.pi/180 # half angle of the hexagone 
density = 0.5 # relative density (roh*/roh_s) of the honeycomb

sampleThickness = 30 # mm

h1 = np.cos(theta)*l
h2 = abs(np.sin(theta)*l)

# t = density*(2*h1*(h/l + abs(np.sin(theta))))/(h/l + 2)




sampleThickness = 30


g = open("seedList.data", 'r', encoding="iso-8859-15") # open data file

seedList = [[1,1,1]]


seedList = np.genfromtxt(g, delimiter = '\t')

box = BasicGeometry(number=0,
                    shape='box',
                    xc=0, yc=0, zc=0,
                    psi=0, theta=0, phi=0,
                    param_geom={"dim_x": sampleThickness,
                                "dim_y": 60,
                                "dim_z": 60})

phases = []
for seed in seedList:
    poly = BasicGeometry(number=0,
                         shape='extrudedpolygon',
                         xc=seed[0] - sampleThickness, yc=seed[1], zc=seed[2],
                         psi=0, theta=0, phi=0,
                         param_geom={"listCorners": [(0, h2 + h/2), (h1, h/2), 
                                                     (h1, -h/2), (0, -h2 - h/2), 
                                                     (-h1, -h/2), (-h1, h/2), 
                                                     (0, h2 + h/2)],
                                     "height": sampleThickness})
    phases.append(poly.generate())
    

# generate CAD geometry
denseSample = box.generate()

sample = cutPhaseByShapeList(phaseToCut=denseSample, cqShapeList=phases)
show(sample[0])

# Octet-Truss

In [ ]:
NPhases_file = 'test_octet.dat'

dt = np.dtype([('number', int), ('shape', np.str_, 10),
               ('xc', np.float64), ('yc', np.float64), ('zc', np.float64),
               ('psi', np.float64), ('theta', np.float64), ('phi', np.float64),
               ('a1', np.float64), ('a2', np.float64)])
# précision du type des données
number, shape, xc, yc, zc, psi, theta, phi, a1, a2, = np.loadtxt(NPhases_file, dtype=dt,
                                                                 usecols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
                                                                 skiprows=1, unpack=True, ndmin=1)

# sections = read_sections(path_data,section_file)

# Size of the mesh
size_mesh = 0.03
a = 1.0
b = 1.0
c = 1.0

revel = Rve(a, b, c, size_mesh)
listPhases = []
listPeriodicPhases = []
n = len(xc)

for i in range(0, n):
    elem = BasicGeometry(number=number[i], shape=shape[i],
                         xc=xc[i], yc=yc[i], zc=zc[i],
                         psi=psi[i], theta=theta[i], phi=phi[i],
                         param_geom={"height": a1[i],
                                     "radius": a2[i]},
                         path_data='')
    listPhases.append(elem.generate())

for phase_elem in listPhases:
    print(phase_elem)
    periodicPhase = periodic(cqshape=phase_elem, rve=revel)
    listPeriodicPhases.append(periodicPhase)

phases_cut = cutParts(cqShapeList=[s[0] for s in listPeriodicPhases], reverseOrder=False)
compound = cq.Compound.makeCompound(phases_cut[0])
show(compound)

# Periodic geometry

In [ ]:
def repeatGeometry(unit_geom, rve, grid):
    import time

    start = time.time()

    xyz_repeat = cq.Assembly()
    for i_x in range(grid["x"]):
        for i_y in range(grid["y"]):
            for i_z in range(grid["z"]):
                xyz_repeat.add(unit_geom, 
                               loc=cq.Location(cq.Vector(i_x*rve.dim_x, 
                                                         i_y*rve.dim_y, 
                                                         i_z*rve.dim_z)))


    end = time.time()

    print(end - start)
    
    return xyz_repeat.toCompound()


final_geom = repeatGeometry(compound, revel, grid={"x": 3, 
                                                   "y": 3, 
                                                   "z": 3})

show(final_geom)

# Gyroid TPMS

In [ ]:
elem = BasicGeometry(number=0, shape='tpms',
                     xc=0.5, yc=0.5, zc=0.5,
                     psi=0., theta=0., phi=0.,
                     param_geom={"type_surface": 'gyroid',
                                 "type_part": 'sheet',
                                 "thickness": 0.1},
                     path_data='data')
# elem.geometry.createSurfaces(rve=revel,
#                              sizeMesh=0.03, minFacetAngle=20., maxRadius=0.03,
#                              path_data='data')
# part = elem.generate(rve=revel)

unit_geom = cq.importers.importStep('gyroid.step')
show(part)

In [ ]:
final_geom = repeatGeometry(part, revel, grid={"x": 3, 
                                               "y": 3, 
                                               "z": 3})
show(final_geom)

# Raster shape

In [ ]:
elem = BasicGeometry(number=101, shape='ellipsoid',
                     xc=0.5, yc=0.5, zc=0.5,
                     psi=0, theta=0, phi=0,
                     param_geom={"a_x": 0.4,
                                 "a_y": 0.25,
                                 "a_z": 0.3},
                     path_data='')
elli = elem.generate(rve=revel)

raster = rasterShapeList(cqShapeList=[elli], rve=revel, grid=[5, 5, 5])

compound = cq.Compound.makeCompound(raster[0])
show(compound)